<a href="https://colab.research.google.com/github/kasobol/Lab_3_sem_3/blob/master/AnalisDataTrades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
# Загрузка данных

import numpy as np
import sklearn
import gc
from sklearn.model_selection import train_test_split

data = []
with open("/content/drive/MyDrive/ColabNotebooks/Data_Trades/XBT_Price_Jump_Min(0.5)_Data_(2017_11_01).txt", 'r') as fObj:
  data = np.array(fObj.read().split()).astype(float)

#data = data[10000000:len(data)]
data = data[:]
x_data = np.array([data[i: i + 50] for i in range(0, len(data) - 59)])
y_data = np.array([np.sum(data[i: i + 10]) for i in range(50, len(data) - 9)])
#y_data = data[50:len(data)]

delimit = int(0.8 * len(x_data))

train_data = x_data[:delimit]
train_targets = y_data[:delimit]
test_data = x_data[delimit:]
test_targets = y_data[delimit:]

#train_data, test_data, train_targets, test_targets = train_test_split(x_data, y_data, test_size = 0.2, shuffle = True)
del x_data
del y_data
del data
gc.collect()


0

In [3]:
mas = np.abs(test_targets)
mas.mean()

27.038989976103995

In [4]:
# Нормализация данных

from sklearn import preprocessing

train_data = preprocessing.normalize(train_data)
test_data = preprocessing.normalize(test_data)

#mean = train_data.mean(axis=0)
#train_data -= mean
#std = train_data.std(axis=0)
#train_data /= std
#test_data -= mean
#test_data /= std

In [5]:
# Конструирование сети

from keras import models
from keras import layers
#from tensorflow.keras.optimizers import Adam

def build_model():
  model = models.Sequential()
  model.add(layers.Dense(50, activation='relu',
  input_shape=(train_data.shape[1],)))
  model.add(layers.Dense(50, activation='relu'))
  model.add(layers.Dense(1))
  optimizer='rmsprop'
  #optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
  model.compile(optimizer = optimizer, loss='mae', metrics=['mae'])
  return model

In [6]:
test_data.shape

(2124623, 50)

In [7]:
test_targets.shape

(2124623,)

In [8]:
num_epochs = 2

model = build_model()

history = model.fit(train_data, train_targets,
 validation_data = (test_data, test_targets),
 epochs=num_epochs, batch_size = 256, verbose = 1)

Epoch 1/2
33198/33198 [==============================] - 65s 2ms/step - loss: 9.7612 - mae: 9.7612 - val_loss: 26.9787 - val_mae: 26.9787
Epoch 2/2
33198/33198 [==============================] - 62s 2ms/step - loss: 9.7042 - mae: 9.7042 - val_loss: 26.9661 - val_mae: 26.9661


In [9]:
mas = model(test_data)

In [11]:
test_targets

array([  0. , -11.5,  -3.5, ...,  11.5,  -4.5,  -3.5])

In [12]:
import tensorflow as tf
mas = tf.reshape(mas, [-1]).numpy()

In [13]:
mas

array([ 4.4742146, -0.3407599,  3.1708238, ..., -3.9761286, -3.688902 ,
       -1.7415874], dtype=float32)

In [16]:
count1 = 0
all_count1 = len(mas)
for i in range(0, len(mas)):
  if mas[i] >= 0 and test_targets[i] >= 0:
    #all_count = all_count + 1
    count1 = count1 + 1
  elif mas[i] < 0 and test_targets[i] < 0:
    #all_count = all_count + 1
    count1 = count1 + 1

In [17]:
count1 / all_count1

0.5362127775139401

In [18]:
count = 0
all_count = 0
for i in range(0, len(mas)):
  if mas[i] >= 0:
    all_count = all_count + mas[i]
    count = count + test_targets[i]
  else:
    all_count = all_count - mas[i]
    count = count - test_targets[i]

In [19]:
count / all_count

0.9696374133812368

In [77]:
num = 0
for i in range(0, len(test_targets)):
  if test_targets[i] >= 0:
    num = num + test_targets[i]
  else:
    num = num - test_targets[i]

In [78]:
num

8742337.700000007

In [76]:
test_targets.shape

(186942,)